## Задача - вычленить из резюме данные в кратком формате, чтобы рекрутер мог увидеть основную информацию о соискателе без прочтения резюме полностью

spaCy имеет предобученные модели для задачи NER, а также инструменты для работы с текстом. Основная идея здесь - добавить в пайплайн обработки текста собственные правила на основе регулярных выражений, которые будут искать такую информацию, как email, ФИО, данные об образовании. После чего в пайплане будет использована обученная на собственных данных (данные парсились с superjob, были размечены для NER, анонимизированы (ФИО, контактные данные генерировались при помощи python библиотек)) модель, которая будет размечать в тексте более сложную информацию, например, об опыте работы кандидата (например, с какого по какой года где работал кандидат, какую должность занимал, какие задачи выполнял). Небольшое заключение о возможных способах применения полученных результатов приведено в конце нотбука.

In [ ]:
import re
import spacy

In [94]:
import pandas as pd

In [95]:
import random
import warnings
warnings.filterwarnings('ignore')
from spacy.util import minibatch, compounding

In [3]:
nlp = spacy.load("en_core_web_lg")

In [96]:
skills = "D:\Python\Diplom\jz_skill_patterns.jsonl"

In [97]:
ruler = nlp.add_pipe("entity_ruler", before="ner")

ValueError: [E007] 'entity_ruler' already exists in pipeline. Existing names: ['tok2vec', 'morphologizer', 'parser', 'senter', 'entity_ruler', 'ner', 'attribute_ruler', 'lemmatizer']

In [6]:
ruler.from_disk(skills)

In [7]:
patterns = [
    {"label":"EMAIL", "pattern":[{"TEXT": {"REGEX":"([^@|\s]+@[^@]+\.[^@|\s]+)"}}]},
    {"label":"MOBILE", "pattern":[{"TEXT": {"REGEX":"\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}"}}]},
]

In [8]:
ruler.add_patterns(patterns)

In [9]:
import docx

In [98]:
doco = docx.Document("D:\\Python\\Diplom\\Resume.docx")
text = ""
all_paras = doco.paragraphs
for para in all_paras:
    text += str(para.text)
tx = " ".join(text.split("\n")) 

In [99]:
doc = nlp(tx)
displacy.render(doc, style="ent", jupyter=True)

In [20]:
from spacy import displacy

In [100]:
dic = {}
skillls = []
i = 0
for ent in doc.ents:
    if ent.label_ == "PERSON" and i == 0: 
        dic["PERSON"] = ent.text
        i = i + 1
    if ent.label_ == "EMAIL":
        dic["EMAIL"] = ent.text
    if ent.label_ == "MOBILE":
        dic["MOBILE"] = ent.text 
    if ent.label_ == "SKILL":
        skillls.append(ent.text)

skillls = [i.capitalize() for i in set([i.lower() for i in skillls])]
dic["SKILLS"] = skillls

In [101]:
dic

{'EMAIL': 'maria123@gmail.com',
 'SKILLS': ['Api',
  'Deployment',
  'Pycharm',
  'Queue',
  'Xml',
  'Algorithms',
  'Data exchange',
  'Data management',
  'Windows',
  'Javascript',
  'Tensorflow',
  'Eclipse',
  'Front end',
  'Django',
  'Pytorch',
  'Databases',
  'Support',
  'Oracle',
  'Java',
  'Html5',
  'Rabbitmq',
  'Shell',
  'Visual studio',
  'Database',
  'C++',
  'Software',
  'Sql',
  'Sqlalchemy',
  'Bootstrap',
  'Json',
  'Design',
  'Testing',
  'Ajax',
  'Business',
  'Html',
  'Debugging',
  'Server',
  'Monitoring',
  'Google',
  'Mysql',
  'Linux',
  'Libraries',
  'Numpy',
  'Python',
  'Sqlite',
  'Pandas',
  'Languages',
  'Css',
  'Angular 2',
  'Code coverage',
  'Php',
  'Framework',
  'Jquery']}

In [33]:
nlp = spacy.load("ru_core_news_lg")

In [34]:
ruler = nlp.add_pipe("entity_ruler", before="ner")

In [35]:
ruler.from_disk(skills)

In [36]:
ruler.add_patterns(patterns)

In [102]:
doco = docx.Document("D:\\Python\\Diplom\\К.docx")
text = ""
all_paras = doco.paragraphs
for para in all_paras:
    text += str(para.text)
tx = " ".join(text.split("\n")) 

In [103]:
doc = nlp(tx)
displacy.render(doc, style="ent", jupyter=True)

In [104]:
dic = {}
skillls = []
i = 0
for ent in doc.ents:
    if ent.label_ == "PERSON" and i == 0: 
        dic["PERSON"] = ent.text
        i = i + 1
    if ent.label_ == "EMAIL":
        dic["EMAIL"] = ent.text
    if ent.label_ == "MOBILE":
        dic["MOBILE"] = ent.text 
    if ent.label_ == "SKILL":
        skillls.append(ent.text)

skillls = [i.capitalize() for i in set([i.lower() for i in skillls])]
dic["SKILLS"] = skillls

In [105]:
dic

{'EMAIL': 'culch1945@gmail.com',
 'MOBILE': '+7(986)1177845Соискатель',
 'SKILLS': ['Adobe photoshop']}

In [55]:
test = pd.read_json("my_cleaned_res.json")
train_data = []
for i in range(160):
    train_data.append([test[0][i],test[1][i]])
train_data

[['Артур Валериевич Гаврюшев. Контакты: culch1945@gmail.com, +7(986)1177845\r\nСоискатель скрыл дату рождения, высшее образование, cостоит в браке, есть дети\r\nРостов-на-Дону, готов\xa0к\xa0переезду: Москва, Санкт-Петербург\r\nГражданство: Россия\r\nТехнический писатель\r\nПолная занятость, удаленная работа, готов к\xa0командировкам\r\nПо договорённости\r\nОпыт работы 41 год\xa0и\xa08 месяцев\r\nМай\xa02021 – ноябрь\xa02021\r\n7 месяцев\r\nТехнический писатель (удаленно)\r\nООО "Квант", Зеленоград\r\nhttp://tvkvant.ru\r\nРазработка технологического оборудования для производства микрочипов.\r\nОбязанности:\r\nОформление пояснительных записок: технического предложения, эскизного проекта, технического проекта разрабатываемого комплекса технологического оборудования для производства микрочипов. Выполнение расчетов планируемых показателей надежности компонентов комплекса.\r\nДостижения:\r\nПояснительные записки, оформленные в соответствии с ГОСТ 2.105-2019 приняты заказчиком без замечаний.

In [51]:
ner=nlp.get_pipe("ner")

In [56]:
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [57]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [62]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training.example import Example

In [63]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(train_data)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        #texts, annotations = zip(*batch)
        nlp.update(
                    example,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

Losses {'ner': 3218.250312805176}
Losses {'ner': 7089.293720245361}
Losses {'ner': 8624.941919326782}
Losses {'ner': 10744.571216583252}
Losses {'ner': 11727.251591682434}
Losses {'ner': 12829.770524024963}
Losses {'ner': 13688.269588470459}
Losses {'ner': 14769.58487701416}
Losses {'ner': 15913.85788345337}
Losses {'ner': 17693.154096603394}
Losses {'ner': 19157.011703252792}
Losses {'ner': 21615.217555761337}
Losses {'ner': 22548.307270407677}
Losses {'ner': 23074.659220576286}
Losses {'ner': 24597.51749098301}
Losses {'ner': 25182.401116788387}
Losses {'ner': 25641.843541890383}
Losses {'ner': 26115.759645074606}
Losses {'ner': 27416.579252809286}
Losses {'ner': 28315.147343724966}
Losses {'ner': 28824.8074541986}
Losses {'ner': 30857.846772283316}
Losses {'ner': 32129.86129102111}
Losses {'ner': 33354.98511657119}
Losses {'ner': 34487.93730315566}
Losses {'ner': 36626.152532190084}
Losses {'ner': 37530.020528405905}
Losses {'ner': 39696.71883830428}
Losses {'ner': 40996.17053470015

Losses {'ner': 20345.431811267626}
Losses {'ner': 20555.36168001846}
Losses {'ner': 321.8066412881017}
Losses {'ner': 757.4897506795824}
Losses {'ner': 1511.2980788312852}
Losses {'ner': 1742.3586085715215}
Losses {'ner': 2025.680719582073}
Losses {'ner': 2459.305487177742}
Losses {'ner': 3600.215573809517}
Losses {'ner': 4099.139460466278}
Losses {'ner': 4997.540739915741}
Losses {'ner': 5620.142930291069}
Losses {'ner': 5962.606540344132}
Losses {'ner': 6658.635159395111}
Losses {'ner': 6959.075091004837}
Losses {'ner': 7800.372347474564}
Losses {'ner': 8040.007662034128}
Losses {'ner': 8663.199442191515}
Losses {'ner': 8988.918566359673}
Losses {'ner': 9368.263694061432}
Losses {'ner': 9802.291948928032}
Losses {'ner': 10323.004042162094}
Losses {'ner': 11020.560672892723}
Losses {'ner': 11310.422246853355}
Losses {'ner': 11534.095373840537}
Losses {'ner': 11873.175362856593}
Losses {'ner': 12035.954488930758}
Losses {'ner': 12626.95911806589}
Losses {'ner': 12931.535844692495}
Loss

Losses {'ner': 11591.142511792725}
Losses {'ner': 11721.596084876695}
Losses {'ner': 12017.568288429418}
Losses {'ner': 125.15443658828735}
Losses {'ner': 329.41033242613776}
Losses {'ner': 505.4743287113379}
Losses {'ner': 674.9630466780509}
Losses {'ner': 816.7439173503153}
Losses {'ner': 1350.7034992022745}
Losses {'ner': 1693.8219734433878}
Losses {'ner': 2158.9566853087235}
Losses {'ner': 2681.183995608597}
Losses {'ner': 2795.961436133897}
Losses {'ner': 2991.451611132252}
Losses {'ner': 3426.6102381654855}
Losses {'ner': 3752.5335357271906}
Losses {'ner': 4354.066683896708}
Losses {'ner': 4458.2106226710675}
Losses {'ner': 4921.263249042451}
Losses {'ner': 5257.080675002195}
Losses {'ner': 5453.939100895084}
Losses {'ner': 5647.078398645863}
Losses {'ner': 5859.939628985252}
Losses {'ner': 6094.352450496059}
Losses {'ner': 6356.970609766089}
Losses {'ner': 6867.347305317067}
Losses {'ner': 7095.148839304289}
Losses {'ner': 7452.784711191496}
Losses {'ner': 7589.050561194839}
Los

Losses {'ner': 8707.138533138084}
Losses {'ner': 8796.887704030141}
Losses {'ner': 9429.367051735982}
Losses {'ner': 109.90443111211061}
Losses {'ner': 275.58854625714594}
Losses {'ner': 358.6447967029453}
Losses {'ner': 573.3374426103474}
Losses {'ner': 825.9158944543378}
Losses {'ner': 955.5517745923062}
Losses {'ner': 1025.489524833807}
Losses {'ner': 1260.4499563326317}
Losses {'ner': 1569.022420917936}
Losses {'ner': 1812.186307483651}
Losses {'ner': 1855.9464468044093}
Losses {'ner': 2159.355619079804}
Losses {'ner': 2295.397073765656}
Losses {'ner': 2778.8503300150837}
Losses {'ner': 2889.08303124029}
Losses {'ner': 3052.3451854618247}
Losses {'ner': 3237.178820881432}
Losses {'ner': 3416.050501224024}
Losses {'ner': 4469.288430568202}
Losses {'ner': 4632.40614121456}
Losses {'ner': 4775.313414060067}
Losses {'ner': 4918.333487994234}
Losses {'ner': 5108.944660926783}
Losses {'ner': 5389.622851992572}
Losses {'ner': 5513.1433410612735}
Losses {'ner': 5570.311294299176}
Losses {'

Losses {'ner': 7136.341553699563}
Losses {'ner': 7255.487455047251}
Losses {'ner': 7321.747365367765}
Losses {'ner': 151.91690962016582}
Losses {'ner': 271.38403157517314}
Losses {'ner': 390.4031369294971}
Losses {'ner': 592.0560318510979}
Losses {'ner': 661.8802946757901}
Losses {'ner': 747.689299084103}
Losses {'ner': 890.7303555606364}
Losses {'ner': 998.1323274373741}
Losses {'ner': 1266.7070411964983}
Losses {'ner': 1363.8272674178756}
Losses {'ner': 1476.6762982523237}
Losses {'ner': 1561.09799598755}
Losses {'ner': 1635.245571588785}
Losses {'ner': 1687.814226627503}
Losses {'ner': 1862.2793916696432}
Losses {'ner': 1970.5582023048523}
Losses {'ner': 2115.907977840838}
Losses {'ner': 2286.9726718393836}
Losses {'ner': 2371.265568130159}
Losses {'ner': 2421.497872526233}
Losses {'ner': 2559.1693047506847}
Losses {'ner': 2670.1169935515204}
Losses {'ner': 2846.415812996594}
Losses {'ner': 3187.5844522346883}
Losses {'ner': 3230.4009433526667}
Losses {'ner': 3328.2761795221018}
Los

Losses {'ner': 4925.887473178287}
Losses {'ner': 5012.515020479086}
Losses {'ner': 5061.598610759114}
Losses {'ner': 5153.42896074819}


In [64]:
nlp.to_disk("tuned_spacy_ner_rus")

In [66]:
nlp_updated = spacy.load("tuned_spacy_ner_rus")
doc = nlp_updated(train_data[0][0])
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Дарья Егоровна Ротманова', 'NAME'), ('daleth2002@yandex.ru', 'EMAIL'), ('+7(962)7738286', 'MOBILE'), ('40 лет', 'AGE'), ('высшее образование', 'EDUCATION'), ('Москва', 'CITY'), ('Менеджер интернет-магазина, товаровед, менеджер по продажам, оператор ПК, продавец-консультант, кассир', 'VACANCY'), ('Не готова к\xa0командировкам', 'WORKCONDITIONS'), ('По договорённости', 'SALARY'), ('13 лет\xa0и\xa06 месяцев', 'WOEXP'), ('Сентябрь\xa02017', 'WORKFROM'), ('май\xa02018', 'WORKTO'), ('Кладовщик, оператор склада', 'POSITION'), ('ООО "Бронницкий Ювелир", Москва', 'ORG'), ('Работа в 1С, 1С8,', 'SUMMARY'), ('CRM', 'SKILL'), ('Январь\xa02012', 'WORKFROM'), ('май\xa02015', 'WORKTO'), ('Менеджер онлайн-продаж', 'POSITION'), ('ОАО «МЮЗ» (Московский ювелирный завод), Москва', 'ORG'), ('Обработка и ведение интернет-заказов, приём звонков, консультации, работа с клиентами по телефону, взаимодействие с курьерскими службами, работа на складе интернет-магазина, взаимодействие с розничными магаз

In [ ]:
doco = docx.Document("D:\\Python\\Diplom\\К.docx")
text = ""
all_paras = doco.paragraphs
for para in all_paras:
    text += str(para.text)
tx = " ".join(text.split("\n")) 

In [67]:
doc = nlp_updated(tx)
displacy.render(doc, style="ent", jupyter=True)

In [70]:
doc

Дарья Егоровна Ротманова. Контакты: daleth2002@yandex.ru, +7(962)7738286
40 лет (родилась 18 апреля 1981), высшее образование, не состоит в браке, детей нет
Москва
Гражданство: Россия
Менеджер интернет-магазина, товаровед, менеджер по продажам, оператор ПК, продавец-консультант, кассир
Не готова к командировкам
По договорённости
Опыт работы 13 лет и 6 месяцев
Сентябрь 2017 – май 2018
9 месяцев
Кладовщик, оператор склада
ООО "Бронницкий Ювелир", Москва
http://www.bronnitsy.com
Обязанности:
Работа в 1С, 1С8, CRM, приём товара, проведение его в 1С. Снятие изделий со склада-запаса, распределение и оформление для отправки в салоны, на дом, Спецсвязь, в сборку. Проверка изделий на брак. Оформление документации, упаковка и биркование изделий. Отправка изделий с курьерами, водителями, взаимодействие с менеджерами интернет-магазина.
Январь 2012 – май 2015
3 года и 5 месяцев
Менеджер онлайн-продаж
ОАО «МЮЗ» (Московский ювелирный завод), Москва
Обязанности:
Обработка и ведение интернет-заказов, п

In [68]:
doc = nlp_updated(train_data[0][0])
displacy.render(doc, style="ent", jupyter=True)

Таким образом, была обучена модель spaCy для задачи Named Entity Recognition для извлечения данных о соискателе на русском языке. Выжные данные имеют свои метки, например, Summary - метка, которая присваивается информации об обязанностях соискателя на одном из предыдущих мест работы, а метка EDUCATION присваивается информации об образовании соискателя. Далее эти данные записывались в MongoDB. В веб-приложении рекрутер мог сначала посмотреть главную информацию из резюме, например навыки кандидата, образование, увидеть контакты, и, если кандидат показался интересным, можно было посмотреть полное резюме. Данный способ позволяет сократить затраты времени на скрининг резюме, а также полученные данные можно в будущем использовать для создания системы, которая на основе ИИ будет сама отбирать наиболее релевантные вакансии резюме.